In [1]:
import OpenAI from "npm:openai";
import { observeOpenAI } from "npm:langfuse";

Deno.env.set("OPENAI_API_KEY", "");
Deno.env.set("LANGFUSE_PUBLIC_KEY", "");
Deno.env.set("LANGFUSE_SECRET_KEY", "");


const openai = observeOpenAI(new OpenAI(), { generationName: "OpenAI.Chat.Trace", tags: ["simple"]} );

// simple

const res = await openai.chat.completions.create({
  model: 'gpt-3.5-turbo',
  messages: [{ role: "system", content: "Tell me a bad joke." }],
  max_tokens: 100,
});

console.log(res.choices[0]?.message.content);

// grouping into single trace

// update trace


Why couldn't the bicycle stand up by itself? Because it was two-tired!


In [2]:
// streaming

const openaiWithLangfuse = observeOpenAI(new OpenAI(), { generationName: "OpenAI.Stream.Trace", tags: ["stream"]} )
const stream = await openaiWithLangfuse.chat.completions.create({
  model: 'gpt-3.5-turbo',
  messages: [{ role: "system", content: "Tell me a funny joke." }],
  stream: true,
});

for await (const chunk of stream) {
    const content = chunk.choices[0]?.delta?.content || '';
    console.log(content);
  }

await openaiWithLangfuse.flushAsync();


Why
 did
 the
 scare
crow
 win
 an
 award
?


Because
 he
 was
 outstanding
 in
 his
 field
!



[
  {
    id: "da606dbe-e421-444c-882d-bf60346bc388",
    type: "trace-create",
    timestamp: "2024-04-16T16:38:43.543Z",
    body: {
      id: "73d17fbd-b4f5-4a14-84c0-8fd34ca7d6ad",
      release: undefined,
      generationName: "OpenAI.Chat.Trace",
      tags: [ "simple" ],
      model: "gpt-3.5-turbo",
      input: { messages: [ [Object] ] },
      modelParameters: {
        frequency_penalty: undefined,
        logit_bias: undefined,
        logprobs: undefined,
        max_tokens: 100,
        n: undefined,
        presence_penalty: undefined,
        seed: undefined,
        stop: undefined,
        stream: undefined,
        temperature: undefined,
        top_p: undefined,
        user: undefined,
        response_format: undefined,
        top_logprobs: undefined
      },
      name: "OpenAI.Chat.Trace",
      startTime: 2024-04-16T16:38:43.543Z,
      timestamp: 2024-04-16T16:38:43.543Z
    },
    metadata: undefined
  },
  {
    id: "598ed629-6a84-4c76-8b32-95863706b342",

In [3]:
// function calling

const openaiWithLangfuse = observeOpenAI(new OpenAI(), { generationName: "OpenAI.Function.Trace", tags: ["function"]} )

async function getWeather(location: string) {
  if (location === "Berlin") {return "20degC"} else {return "unknown"}
}

const functions = [{
            type: "function",
            function: {
              name: "getWeather",
              description: "Get the current weather in a given location",
              parameters: {
                type: "object",
                properties: {
                  location: {
                    type: "string",
                    description: "The city, e.g. San Francisco",
                  },
                },
                required: ["location"],
              },
            },
          }]

// Main function to execute the OpenAI chat completions and auxiliary functions
async function main() {
      const messages =  [{
              role: 'user',
              content:
                "What's the weather like in Berlin today",
            },
          ]
      const res = await openaiWithLangfuse.chat.completions.create({
          model: 'gpt-3.5-turbo',
          messages: messages,
          tool_choice: "auto",
          tools: functions,
        })
        const content = res.choices[0].message.content;
        const tool_call = res.choices[0].message.tool_calls;
        if (tool_call[0].function.name === "getWeather") {
            const argsStr = tool_call[0].function.arguments;
            const args = JSON.parse(argsStr); 
            const answer = await getWeather(args["location"]);
        }
}

main();
await openaiWithLangfuse.flushAsync();

[
  {
    id: "c36ae10d-1bea-4a13-9d02-a07b644a42bb",
    type: "trace-create",
    timestamp: "2024-04-16T16:38:45.282Z",
    body: {
      id: "88ecc5ea-43c9-48c6-8fc5-11273596d652",
      release: undefined,
      generationName: "OpenAI.Function.Trace",
      tags: [ "function" ],
      model: "gpt-3.5-turbo",
      input: {
        messages: [ [Object] ],
        tools: [ [Object] ],
        tool_choice: "auto"
      },
      modelParameters: {
        frequency_penalty: undefined,
        logit_bias: undefined,
        logprobs: undefined,
        max_tokens: undefined,
        n: undefined,
        presence_penalty: undefined,
        seed: undefined,
        stop: undefined,
        stream: undefined,
        temperature: undefined,
        top_p: undefined,
        user: undefined,
        response_format: undefined,
        top_logprobs: undefined
      },
      name: "OpenAI.Function.Trace",
      startTime: 2024-04-16T16:38:45.282Z,
      timestamp: 2024-04-16T16:38:45.282Z

In [4]:
// grouped
import Langfuse from "npm:langfuse";


const langfuse = new Langfuse();
const openai = new OpenAI();
 
// Create trace and add params
const trace = langfuse.trace({ name: "capital-poem-generator" });
 
// Create span
const country = "Germany";
const span = trace.span({ name: country });

const capital = (
  await observeOpenAI(openai, {
    parent: span,
    generationName: "get-capital",
    tag: ["grouped"],
  }).chat.completions.create({
    model: "gpt-3.5-turbo",
    messages: [
      { role: "system", content: "What is the capital of the country?" },
      { role: "user", content: country },
    ],
  })
).choices[0].message.content;

const poem = (
  await observeOpenAI(openai, {
    parent: span,
    generationName: "generate-poem",
  }).chat.completions.create({
    model: "gpt-3.5-turbo",
    messages: [
      {
        role: "system",
        content: "You are a poet. Create a poem about this city.",
      },
      { role: "user", content: capital },
    ],
  })
).choices[0].message.content;

span.end();
 
// Flush the Langfuse client belonging to the parent span
await langfuse.flushAsync();

[
  {
    id: "09de8052-6fe2-4ad8-afea-bc49a968fe6e",
    type: "trace-create",
    timestamp: "2024-04-16T16:38:45.464Z",
    body: {
      id: "df08c671-d694-4345-ad5c-1b74316127e6",
      release: undefined,
      name: "capital-poem-generator"
    },
    metadata: undefined
  },
  {
    id: "b4deb7e1-92ac-4f98-bc84-26b5e4ab2058",
    type: "span-create",
    timestamp: "2024-04-16T16:38:45.465Z",
    body: {
      id: "581aa84d-0ee3-47b4-b78d-6fecabd7206c",
      startTime: 2024-04-16T16:38:45.465Z,
      name: "Germany",
      traceId: "df08c671-d694-4345-ad5c-1b74316127e6",
      parentObservationId: null
    },
    metadata: undefined
  },
  {
    id: "0e5bd7fb-e794-428b-9a30-87f96e340caa",
    type: "generation-create",
    timestamp: "2024-04-16T16:38:45.937Z",
    body: {
      id: "5656aeb5-9a1b-4c9f-bc56-ee2115abc855",
      startTime: 2024-04-16T16:38:45.465Z,
      parent: LangfuseSpanClient {
        client: Langfuse {
          debugMode: false,
          pendingPromise

In [6]:
// Update trace example

const trace = langfuse.trace({ name: "capital-poem-generator" });

const span = trace.span({ name: "France" });

const capital = (
  await observeOpenAI(openai, {
    parent: span,
    generationName: "get-capital",
    tag: ["update"],
  }).chat.completions.create({
    model: "gpt-3.5-turbo",
    messages: [
      { role: "system", content: "What is the capital of the country?" },
      { role: "user", content: "France" },
    ],
  })
).choices[0].message.content;

const poem = (
  await observeOpenAI(openai, {
    parent: span,
    generationName: "generate-poem",
    tag: ["update"],
  }).chat.completions.create({
    model: "gpt-3.5-turbo",
    messages: [
      {
        role: "system",
        content: "You are a poet. Create a poem about this city.",
      },
      { role: "user", content: capital },
    ],
  })
).choices[0].message.content;

span.update({input: capital, output: poem});
span.end();

trace.update({name:"City poem generator", tags: ["updated"], metadata: {"env": "development"}, release: "v0.0.21"});

await langfuse.flushAsync();

[
  {
    id: "780f3812-5595-4508-b667-653a2bb70dbe",
    type: "trace-create",
    timestamp: "2024-04-16T16:42:25.612Z",
    body: {
      id: "97e90fab-94d5-4f50-b8ca-f4b63018d4eb",
      release: undefined,
      name: "capital-poem-generator"
    },
    metadata: undefined
  },
  {
    id: "eb715898-fe3e-4813-868d-ff14a0844bc1",
    type: "span-create",
    timestamp: "2024-04-16T16:42:25.613Z",
    body: {
      id: "7c885f0b-76f1-45c8-9c53-38d09751c9ae",
      startTime: 2024-04-16T16:42:25.613Z,
      name: "France",
      traceId: "97e90fab-94d5-4f50-b8ca-f4b63018d4eb",
      parentObservationId: null
    },
    metadata: undefined
  },
  {
    id: "f2b7d771-c218-46e4-809d-a26c5787d346",
    type: "generation-create",
    timestamp: "2024-04-16T16:42:26.267Z",
    body: {
      id: "68e2bf2a-4126-4cf4-bd80-1fbe05944137",
      startTime: 2024-04-16T16:42:25.613Z,
      parent: LangfuseSpanClient {
        client: Langfuse {
          debugMode: false,
          pendingPromises